In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate invisible_watermark mediapy omegaconf gradio

In [ ]:
!pip install omegaconf


In [ ]:
use_refiner = False

In [ ]:
#@title Setup Model

import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline

model = "digiplay/Realisian_v6" 

pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
)

pipe = pipe.to("cuda") 


In [ ]:
# @title PROMPT, GUNAKAN BAHASA INGGRIS / USE ENGLISH { display-mode: "form" }
import os
prompt = "garden having lake sideby and buetiful view of the mountains with the snow on it" 
seed = 50000000000

negative_prompt = "bad-picture-chill-75v, ng_deepnegative_v1_75t, badhandv4, (worst quality:2), (low quality:2), (normal quality:2), (lowres:2), (bad anatomy:2), (bad hands:2), (watermark:2), (mole:1.5), (freckles:1.5)" 

width = 1064  
height = 936  
num_image = 1 
similarity2prompt = 7.5 
step = 50

width = int(width)
height = int(height)
image = int(num_image)
guidance = float(similarity2prompt)
step = int(step)

images = pipe(
    prompt=prompt,
    width=width,
    height=height,
    num_images_per_prompt = image,
    guidance_scale = guidance,
    num_inference_steps = step,
    negative_prompt=negative_prompt,
    output_type="latent" if use_refiner else "pil",
    generator=torch.Generator().manual_seed(seed)
).images


if use_refiner:
  images = refiner(
      prompt = prompt,
      negative_prompt = negative_prompt,
      image = images,
      ).images

print(f"Prompt:\t{prompt}\nSeed:\t{seed}")


base_filename = "output.jpg"
new_filename = base_filename


if os.path.exists(base_filename):
    index = 1
    while True:
        new_filename = f"output_{index}.jpg"
        if not os.path.exists(new_filename):
            break
        index += 1

images[0].save(new_filename)

media.show_images(images)

## **TOOLS**

In [ ]:
#@title FIX FALSE NSFW
pipe.safety_checker = None
pipe.requires_safety_checker = False

#JIKA RUNTIME RESTART, JALANKAN KODE INI LAGI
#IF THE RUNTIME RESTARTS, RUN THIS CODE AGAIN